In [1]:
import numpy as np
import pandas as pd

In [2]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
        self.S_train=[]
        self.S_test=[]
        self_folds=[]
        
    def fit(self, X, y):
        self.folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=17))
        self.S_train = np.zeros((X.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            for j, (train_idx, test_idx) in enumerate(self.folds):
                X_train = X.iloc[train_idx]
                y_train = y.iloc[train_idx]
                X_holdout = X.iloc[test_idx]
                # y_holdout = y[test_idx]
                clf[j].fit(X_train, y_train)
                y_pred = clf[j].predict(X_holdout)[:]
                self.S_train[test_idx, i] = y_pred
        self.stacker.fit(self.S_train, y)
        
    def predict(self, X_test):             
        self.S_test = np.zeros((X_test.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((X_test.shape[0], len(self.folds)))
            for j in range(len(self.folds)):
                S_test_i[:, j] = clf[j].predict(X_test)[:]
            self.S_test[:, i] = S_test_i.mean(1)
        y_pred = self.stacker.predict(self.S_test)[:]
        return y_pred